# Distinguishing k-regular graphs


Multiple graph structures are not distinguishable by the standard message
passing Graph Neural Networks.
For instance, the GNNs are not able to distinguish between
_k_-regular graphs of the same size, such as the pair shown in the following image. Those graphs are not isomorphic and are both $3$-regular, meaning all nodes have precisely three neighbors.

![3-Regular Graphs](https://raw.githubusercontent.com/LukasZahradnik/PyNeuraLogic/master/docs/_static/k_regular_graph.png)

Install PyNeuraLogic from PyPI

In [1]:
! pip install neuralogic

In [2]:
from neuralogic.nn import get_evaluator
from neuralogic.core import R, Template, V
from neuralogic.core.settings import Settings
from neuralogic.dataset import Dataset
from neuralogic.optim import SGD

C:\Users\kraus\Plocha\BAK\PyNeuraLogicFork\neuralogic\core\builder\builder.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


When we assign the same features to all the nodes, the messages during the
update step of the GNNs will be identical, resulting in
the same node representations and eventually classifying both graphs the same.
Such misclassification can be problematic in multiple domains, e.g.,
chemistry, where two indistinguishable graphs represent two different molecules.

With the PyNeuraLogic library, we are able to express various patterns in graphs
or their parts. There are many alternative approaches to distinguish between
those two graphs. Here we will utilize the previously introduced encoding of triangles to capture
the triangles of graph _b_, and add some extra rules for capturing the
general graph structure in the standard (GNN) fashion.


In [3]:
train_dataset = Dataset()
template = Template()

template.add_rules([
    # Captures triangle
    R.triangle(V.X)[1,] <= (
        R.edge(V.X, V.Y), R.feature(V.Y)[1,],
        R.edge(V.Y, V.Z), R.feature(V.Z)[1,],
        R.edge(V.Z, V.X), R.feature(V.X)[1,],
    ),

    # Captures general graph
    R.general(V.X)[1,] <= (R.edge(V.Y, V.X), R.feature(V.Y)[1,]),
    R.general(V.X)[1,] <= R.feature(V.X)[1,],

    R.predict <= R.general(V.X)[1,],
    R.predict <= R.triangle(V.X)[1,],
])

train_dataset.add_example(
    [
        R.edge(1, 2), R.edge(2, 3), R.edge(3, 4), R.edge(4, 1),
        R.edge(2, 1), R.edge(3, 2), R.edge(4, 3), R.edge(1, 4),
        R.edge(1, 6), R.edge(3, 6), R.edge(4, 5), R.edge(2, 5),
        R.edge(6, 1), R.edge(6, 3), R.edge(5, 4), R.edge(5, 2),

        R.feature(1), R.feature(2), R.feature(3),
        R.feature(4), R.feature(5), R.feature(6),
    ],
)

train_dataset.add_example(
    [
        R.edge(1, 2), R.edge(2, 3), R.edge(3, 4), R.edge(4, 1),
        R.edge(2, 1), R.edge(3, 2), R.edge(4, 3), R.edge(1, 4),
        R.edge(1, 6), R.edge(4, 6), R.edge(3, 5), R.edge(2, 5),
        R.edge(6, 1), R.edge(6, 4), R.edge(5, 3), R.edge(5, 2),

        R.feature(1), R.feature(2), R.feature(3),
        R.feature(4), R.feature(5), R.feature(6),
    ],
)

train_dataset.add_queries([
    R.predict[1],
    R.predict[0],
])

In [4]:
settings = Settings(optimizer=SGD(), epochs=200)
neuralogic_evaluator = get_evaluator(template, settings)

for _ in neuralogic_evaluator.train(train_dataset):
    pass

graphs = ["a", "b"]

for graph_id, predicted in enumerate(neuralogic_evaluator.test(train_dataset)):
    print(f"Graph {graphs[graph_id]} is predicted to be class: {int(round(predicted))} | {predicted}")

Graph a is predicted to be class: 1 | 0.705858168713444
Graph b is predicted to be class: 0 | 0.0580546259618393
